# Implementation notes

## Abstract

The reason for this implementation was to try and perform the classification with discrete data straight from the MIDI files in order to avoid inaccuracies and complications introduced in the processing of the continuous, multi-dimensional features we extracted using **jAudio**.

## Chosen Features

The features used in this algorithm are extracted directly from the MIDI files and were chosen for simplicity's sake. The chosen features are as follows:

### Key Signature

The first key signature given in the MIDI file. Subsequent key signature changes are ignored for simplicity.
Seems to be a strong predictor.

### Time Signature

The first time signature given in the MIDI file. Subsequent time signature changes are ignored since they are fairly uncommon in our dataset.

### Mean Tempo

An average of the tempo changes throughout the whole piece. This is measured in ticks. The mean tempo is then discretized by finding the mean $\mu$ and standard deviation $\sigma^2$ of the mean tempos across all data points and then turning them into discrete values according to the rule:

$T_{class} = 0 \quad$ if $T_{value} < \mu - \sigma^2$, Low tempo.

$T_{class} = 1 \quad$ if $\mu - \sigma^2 <= T_{value} <= \mu + \sigma^2$, Mid tempo.

$T_{class} = 2 \quad$ if $T_{value} < \mu + \sigma^2$, High tempo.


In [32]:
# Initialization Cell
import os
import numpy as np
from random import randint
from scipy.stats import norm
from mido import MidiFile

# Data Extraction

In [33]:
def extractData():
    fileNames = []
    data = []
    for file in os.listdir(os.getcwd() + "/midiFiles"):
        if file.endswith(".mid"):
            fileNames.append(file)
    
    for file in fileNames:
        data.append(MidiFile("midiFiles/" + file))
    return data  

## Labels

In [34]:
def getComposer(mid):
    for track in mid.tracks:
        for msg in track:
            if msg.type == 'text':
                return msg.text

In [35]:
# getComposer(rawData[0])

## Features

In [36]:
def getTimeSig(mid):
    """
    Take in a MIDI file and return a list of time signature changes
    """
    
    
    for track in mid.tracks:
        for msg in track:
            if(msg.type == 'time_signature'):
                return str(msg.numerator) + ',' + str(msg.denominator)
    
    raise(NotImplementedError)

In [37]:
def getKey(mid):
    """
    Take in a midi file and return the key
    """
    
    for track in mid.tracks:
        for msg in track:
            if(msg.type == 'key_signature'):
                return(msg.key)

In [38]:
def getAvgTempo(mid):
    """
    Take in a MIDI file and return the average tempo
    """
    temps = []
    for track in mid.tracks:
        for msg in track:
            if msg.type == 'set_tempo':
                temps.append(msg.tempo)
    return sum(temps) / len(temps)

In [39]:
# getTimeSig(rawData[0])

# Data Preparation

In [45]:
def prepareData(data, accepted_composers):
    y = []
    x_key = []
    x_sig = []
    x_tempo = []
    for song in data:
        c = getComposer(song)
        for rule in accepted_composers:
            if rule in c:
                y.append(rule)
                x_key.append(getKey(song))
                x_sig.append(getTimeSig(song))
                x_tempo.append(getAvgTempo(song))
    
    # Discretize tempo data into low, mid, high
    [mu, std] = norm.fit(x_tempo)
    l1 = mu - std
    l2 = mu + std
    
    for i in range(len(x_tempo)):
        if x_tempo[i] < l1:
            x_tempo[i] = 0 # low tempo
        elif x_tempo[i] > l2:
            x_tempo[i] = 2 # high tempo
        else:
            x_tempo[i] = 1 # mid tempo. about 65% of data should lie in this group
            
    
    return [[x_key, x_sig, x_tempo],y]

In [47]:
# prepareData(rawData, ['Mozart', 'Chopin'])

In [48]:
def splitData(data, labels, ratio):
    n = int(len(data[0])*ratio)
    d_train = []
    l_train = []
    kt = []
    st = []
    for i in range(n):
        x = randint(0, len(labels) - 1)
        kt.append(data[0].pop(x))
        st.append(data[1].pop(x))
        l_train.append(labels.pop(x))
    d_train = [kt, st]
    return [d_train, l_train, data, labels]

In [49]:
# dat = splitData(xVals, yVals, 0.5)

# Learning

In [50]:
# rawData = extractData()
# [xVals, yVals] = prepareData(rawData, problem_space)

In [75]:
def learnNaiveBayes(t_data, t_labels):
    """
    Take in training data and corresponing labels, return a NB model
    Inputs:
        t_data - a list of features, each feature being a list of values
        t_labels - the corresponding labels
    """
    composers = list(set(t_labels))
    comp_counts = []
    tables = []
    py = []
    for composer in composers:
        py.append(t_labels.count(composer) / len(t_labels))
        comp_counts.append(t_labels.count(composer))
        
    for i in range(len(t_data)):
        f_vals = list(set(t_data[i]))
        px_feat = [0]*len(f_vals)
        for j in range(len(f_vals)):
            px_feat[j] = t_data[i].count(f_vals[j]) / len(t_labels)
        pxy_feat = []
        for composer in composers:
            k = []
            for j in range(len(t_labels)):
                if t_labels[i] == composer:
                    k.append(t_data[i][j])
            pxy_feat.append(k)
        feat_table = [f_vals, px_feat]
    
        for j in range(len(composers)):
            k = []
            for m in range(len(f_vals)):
                k.append((pxy_feat[j].count(f_vals[m])) / comp_counts[i])
            feat_table.append(k)
        tables.append(feat_table)
        
    return [composers, comp_counts, py, tables]

In [76]:
# trained_model = learnNaiveBayes(xVals, yVals)

# Classification

In [82]:
def classifySongNB(model, xVal):
    composers = model[0]
    comp_counts = model[1]
    py = model[2]
    probs = [1]*len(composers)
    tables = model[3]
    
    for i in range(len(xVal)):
        tab = tables[i]
        if xVal[i] not in tab[0]:
            break
            px = 1/ sum(comp_counts)
        else:
            j = tab[0].index(xVal[i])
            px = tab[1][j] + 1/ sum(comp_counts)
        
        for m in range(len(composers)):
            pxy = tab[2 + m][j]
        probs[m] *= (pxy * py[m]) / px
    return(composers[probs.index(max(probs))])

In [83]:
# trained_model = learnNaiveBayes(xVals, yVals)
# classifySongNB(trained_model, [xVals[0][0], xVals[1][0]])

# Testing

In [93]:
def testNB(model, t_data, t_labels):
    good = 0
    bad = 0
    n = len(t_labels)
    for i in range(len(t_labels)):
        prediction = classifySongNB(model, [t_data[0][i], t_data[1][i]])
#         print('True: ', t_labels[i], ' ; Predicted: ', prediction)
#         print('')
        if prediction == t_labels[i]:
            good += 1
        else:
            bad += 1
    print('Good: ', good / n, ', Baad: ', bad / n)

In [85]:
problem_space = ['Mozart', 'Chopin'] # ['Mozart', 'Chopin', 'Schubert']
rawData = extractData()
[xVals, yVals] = prepareData(rawData, problem_space)


In [86]:
dat = splitData(xVals, yVals, 0.5)
trained_model = learnNaiveBayes(dat[0], yVals)
testNB(trained_model, dat[2], dat[3])

Good:  0.7352941176470589 , Baad:  0.2647058823529412


In [92]:
# run the test 10 times
# DANGER
problem_space = ['Mozart', 'Chopin']
for i in range(10):
    rawData = extractData()
    [xVals, yVals] = prepareData(rawData, problem_space)
    dat = splitData(xVals, yVals, 0.6)
    trained_model = learnNaiveBayes(dat[0], yVals)
    testNB(trained_model, dat[2], dat[3])

Good:  0.8214285714285714 , Baad:  0.17857142857142858
Good:  0.7857142857142857 , Baad:  0.21428571428571427
Good:  0.7142857142857143 , Baad:  0.2857142857142857
Good:  0.7142857142857143 , Baad:  0.2857142857142857
Good:  0.7857142857142857 , Baad:  0.21428571428571427
Good:  0.6785714285714286 , Baad:  0.32142857142857145
Good:  0.6785714285714286 , Baad:  0.32142857142857145
Good:  0.5357142857142857 , Baad:  0.4642857142857143
Good:  0.75 , Baad:  0.25
Good:  0.7142857142857143 , Baad:  0.2857142857142857


In [91]:
# run the test 10 times
# DANGER
problem_space = ['Mozart', 'Chopin', 'Schubert']
for i in range(10):
    rawData = extractData()
    [xVals, yVals] = prepareData(rawData, problem_space)
    dat = splitData(xVals, yVals, 0.6)
    trained_model = learnNaiveBayes(dat[0], yVals)
    testNB(trained_model, dat[2], dat[3])

Good:  0.48717948717948717 , Baad:  0.5128205128205128
Good:  0.41025641025641024 , Baad:  0.5897435897435898
Good:  0.5897435897435898 , Baad:  0.41025641025641024
Good:  0.4358974358974359 , Baad:  0.5641025641025641
Good:  0.4358974358974359 , Baad:  0.5641025641025641
Good:  0.5641025641025641 , Baad:  0.4358974358974359
Good:  0.5384615384615384 , Baad:  0.46153846153846156
Good:  0.46153846153846156 , Baad:  0.5384615384615384
Good:  0.48717948717948717 , Baad:  0.5128205128205128
Good:  0.48717948717948717 , Baad:  0.5128205128205128


In [ ]:
# problem_space = ['Mozart', 'Chopin'] # ['Mozart', 'Chopin', 'Schubert']
# rawData = extractData()

In [68]:
def getFeat(mid):
    """
    Take in a MIDI file and return a list of time signature changes
    """
    
    
    for track in mid.tracks:
        for msg in track:
            print(msg)
#             if(msg.type == 'time_signature'):
#                 return str(msg.numerator) + ',' + str(msg.denominator)
    
    raise(NotImplementedError)

In [69]:
getFeat(rawData[0])

<meta message track_name name='Vier Impromptus Opus postb. 142 D 935' time=0>
<meta message track_name name='Impromptu Nr. 2 As-Dur' time=0>
<meta message copyright text='Copyright © 2007 von Bernd Krüger.' time=0>
<meta message text text='Franz Schubert ' time=0>
<meta message text text='Allegretto' time=0>
<meta message text text='Erstellt am 15.8.2007\n' time=0>
<meta message text text='Update am 20.3.2014\n' time=0>
<meta message text text='Dauer: 6:24 Minuten\n' time=0>
<meta message time_signature numerator=3 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
<meta message key_signature key='Ab' time=0>
<meta message set_tempo tempo=597015 time=0>
<meta message marker text='Allegretto' time=0>
<meta message set_tempo tempo=597015 time=948>
<meta message set_tempo tempo=545455 time=972>
<meta message set_tempo tempo=600000 time=960>
<meta message set_tempo tempo=545455 time=468>
<meta message set_tempo tempo=600000 time=1932>
<meta message set_tempo tempo=5504

note_on channel=0 note=68 velocity=0 time=475
note_on channel=0 note=63 velocity=0 time=0
note_on channel=0 note=63 velocity=39 time=5
note_on channel=0 note=61 velocity=39 time=0
note_on channel=0 note=70 velocity=52 time=0
note_on channel=0 note=70 velocity=0 time=475
note_on channel=0 note=61 velocity=0 time=0
note_on channel=0 note=63 velocity=0 time=0
note_on channel=0 note=63 velocity=40 time=5
note_on channel=0 note=60 velocity=40 time=0
note_on channel=0 note=72 velocity=55 time=0
note_on channel=0 note=72 velocity=0 time=960
note_on channel=0 note=60 velocity=0 time=0
note_on channel=0 note=63 velocity=0 time=0
note_on channel=0 note=63 velocity=55 time=0
note_on channel=0 note=63 velocity=0 time=475
note_on channel=0 note=72 velocity=66 time=5
note_on channel=0 note=60 velocity=49 time=0
note_on channel=0 note=63 velocity=49 time=0
note_on channel=0 note=63 velocity=0 time=475
note_on channel=0 note=60 velocity=0 time=0
note_on channel=0 note=72 velocity=0 time=0
note_on chan

note_on channel=0 note=73 velocity=0 time=160
note_on channel=0 note=68 velocity=52 time=0
note_on channel=0 note=68 velocity=0 time=160
note_on channel=0 note=73 velocity=48 time=0
note_on channel=0 note=73 velocity=0 time=160
note_on channel=0 note=77 velocity=52 time=0
note_on channel=0 note=77 velocity=0 time=160
note_on channel=0 note=74 velocity=52 time=0
note_on channel=0 note=74 velocity=0 time=160
note_on channel=0 note=77 velocity=49 time=0
note_on channel=0 note=77 velocity=0 time=160
note_on channel=0 note=80 velocity=52 time=0
note_on channel=0 note=80 velocity=0 time=160
note_on channel=0 note=75 velocity=52 time=0
note_on channel=0 note=75 velocity=0 time=160
note_on channel=0 note=78 velocity=49 time=0
note_on channel=0 note=78 velocity=0 time=160
note_on channel=0 note=80 velocity=49 time=0
note_on channel=0 note=80 velocity=0 time=160
note_on channel=0 note=75 velocity=54 time=0
note_on channel=0 note=75 velocity=0 time=160
note_on channel=0 note=78 velocity=57 time=0

note_on channel=0 note=51 velocity=0 time=0
note_on channel=0 note=51 velocity=61 time=2
note_on channel=0 note=56 velocity=72 time=0
note_on channel=0 note=44 velocity=61 time=0
note_on channel=0 note=44 velocity=0 time=475
note_on channel=0 note=56 velocity=0 time=0
note_on channel=0 note=51 velocity=0 time=0
note_on channel=0 note=51 velocity=59 time=5
note_on channel=0 note=56 velocity=70 time=0
note_on channel=0 note=44 velocity=59 time=0
note_on channel=0 note=44 velocity=0 time=713
note_on channel=0 note=56 velocity=0 time=0
note_on channel=0 note=51 velocity=0 time=0
note_on channel=0 note=41 velocity=54 time=7
note_on channel=0 note=53 velocity=64 time=0
note_on channel=0 note=53 velocity=0 time=238
note_on channel=0 note=41 velocity=0 time=0
note_on channel=0 note=41 velocity=54 time=2
note_on channel=0 note=53 velocity=64 time=0
note_on channel=0 note=53 velocity=0 time=360
note_on channel=0 note=41 velocity=0 time=0
note_on channel=0 note=55 velocity=64 time=120
note_on cha

NotImplementedError: 